# The geo sub-package

.. versionadded:: 0.1.9

.. contents::
    :local:

:class:`dimarray.geo.GeoArray` is a subclass of :class:`dimarray.DimArray` that is more specific to geoscientific applications. The most recognizable features are automatic checks for longitude and latitude coordinates.

In [1]:
from dimarray.geo import GeoArray

In [2]:
a = GeoArray([0,0,0], axes=[('lon',[-180,0,180])])
a

geoarray: 3 non-null elements (0 null)
0 / lon (3): -180.0 to 180.0
array([0, 0, 0])

Coordinate axes can now be defined as keyword arguments:

In [3]:
import numpy as np

In [4]:
a = GeoArray(np.ones((2,3,4)), time=[1950., 1960.], lat=np.linspace(-90,90,3), lon=np.linspace(-180,180,4))
a

geoarray: 24 non-null elements (0 null)
0 / time (2): 1950.0 to 1960.0
1 / lat (3): -90.0 to 90.0
2 / lon (4): -180.0 to 180.0
array([[[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]]])

.. note:: The keyword arguments assume an order time (`time`), vertical dimension (`z`), horizontal northing dimension (`lat` or `y`) and  horizontal easting dimension (`x` or `lon`), following CF-recommandations. 

All standard dimarray functions are available under dimarray.geo (so that `import dimarray.geo as da` works), and a few functions or classes such as :func:`read_nc` or :class:`Dataset` are modified to return :class:`GeoArray` instead of :class:`DimArray` instances.

## Coordinate Axes

Under the hood, there are new :class:`Coordinate` classes which inherit from :class:`Axis`.

For example, the inheritance relations of Latitude is: :class:`Latitude` -> :class:`Y` -> :class:`Coordinate` -> :class:`Axis`.

In [5]:
from dimarray.geo import Latitude, Y, Coordinate, Axis

In [6]:
assert isinstance(a.axes['lat'], Latitude) 
assert issubclass(Latitude, Y) 
assert issubclass(Y, Coordinate) 
assert issubclass(Coordinate, Axis)

Weights are automatically defined Latitude axis, so that a mean is weighed by default.

In [7]:
a.axes['lat'].weights  # lat -> cos(lat) weighted mean # doctest: +SKIP

<function dimarray.geo.geoarray.<lambda>>

In the case of Latitude and Longitude, some metadata are also provided by default.

In [8]:
a.axes['lat'].attrs  # doctest: +SKIP

OrderedDict([('units', 'degrees_north'),
             ('long_name', 'latitude'),
             ('standard_name', 'latitude')])

.. note :: For now there is no constraint on the coordinate axis. This might change in the future, by imposing a strict ordering relationship. 

.. seealso:: :ref:`ref_api_geo`

## Projections

dimarray.geo is shipped with :func:`dimarray.geo.transform` and :func:`dimarray.geo.transform_vectors` functions to handle transformations across coordinate reference systems. They are based on :class:`cartopy.crs.CRS`. Cartopy itself makes use of the `PROJ.4` library. In addition to the list of cartopy projections, the :class:`dimarray.geo.crs.Proj4` class makes it possible to define a projection directly from `PROJ.4 parameters <https://trac.osgeo.org/proj/wiki/GenParms>`_. For the most common projections, :mod:`dimarray.geo.crs` also provides wrapper classes that can be initialized with `CF parameters <http://cfconventions.org>`_. See :func:`dimarray.geo.crs.get_crs` for more information.

In contrast to cartopy/PROJ.4, dimarray.geo functions perform both coordinate transforms and regridding onto a regular grid in the new coordinate system. This is because of the structure of DimArray and GeoArray classes, which only accept regular grids (in the sense of a collection of 1-D axes).

.. note :: Why cartopy and not just pyproj? Pyproj would be just fine, and is more minimalistic, but cartopy also implements vector transformas and offers other useful features related to plotting, reading shapefiles, download online data and so on, which come in handy. Moreover it feels more `"pythonic" <http://legacy.python.org/dev/peps/pep-0008>`_, is actively developed with support from the Met' Office, and is related to another interesting project, iris. It builds on other powerful packages such as shapely and it feels like in the long (or not so long) run it might grow toward something even more useful.

.. seealso:: :ref:`projection`